##### Import function test()

In [11]:
from test_kit import test

##### Confirming infections using test()

In [6]:
with open('person_00027.txt', 'r') as file:
    file_content = file.read()
    
test(file_content)

True

In [7]:
with open('person_00017.txt', 'r') as file:
    file_content = file.read()
    
test(file_content)

False

In [8]:
with open('person_00037.txt', 'r') as file:
    file_content = file.read()
    
test(file_content)

False

##### Creating get_genome()

In [12]:
def get_genome(person_id):
    with open(f'person_{person_id:05d}.txt', 'r') as file:
        file_content = file.read()
    return file_content

##### Applying test() to first 100 people

In [3]:
infections = []

for i in range(100):
    genome = get_genome(i)
    result = test(genome)
    
    if test(genome) == True:
        infections.append(i)
            
print(f'Infected people: {infections}')

Infected people: [1, 7, 8, 27, 47, 57, 62, 63, 78]


##### Testing 100 random people to estimate infection rate

In [16]:
import numpy.random as rnd

sample_size = 100

sample = rnd.randint(0, 500, sample_size) 
no_infections = 0

for i in sample:
    genome = get_genome(i)
    result = test(genome)
    
    if result == True:
        no_infections += 1
    
print(f'Estimated infection rate: {no_infections / sample_size * 100:.2f}%')

Estimated infection rate: 8.00%


##### Combining genomes

In [50]:
genome_1 = get_genome(1)
genome_2 = get_genome(2)

combined_genome = genome_1 + genome_2

##### Creating pool()

In [13]:
def pool(list_of_id):
    combined_genomes = ''
    
    for i in list_of_id:
        combined_genomes += get_genome(i)
    
    return combined_genomes

In [43]:
test(pool([20, 21]))

False

In [44]:
test(pool([21, 22]))

False

The results are as expected, because persons 20, 21 and 22 are individually not infected. Hence combining their genomes will also not create an infection.

##### Testing in groups of 10

In [18]:
def pool_sampling(population_size, grouping_size):
    persons = list(range(population_size))
    groups = [persons[i:i+grouping_size] for i in range(0, population_size, grouping_size)]
    infected_list = []
    test_counter = 0

    for i, group in enumerate(groups):
        combined_genome = pool(group)
        group_result = test(combined_genome)
        test_counter += 1
    
        if group_result == True:
            for person in range(grouping_size):
                person_genome = get_genome(groups[i][person])
                test_result = test(person_genome)
                test_counter += 1
            
                if test_result == True:
                    infected_list.append(groups[i][person])

    print(f"Infected people: {infected_list}")
    print(f"Number of tests performed: {test_counter}")

In [19]:
pool_sampling(100, 10)

Infected people: [1, 7, 8, 27, 47, 57, 62, 63, 78]
Number of tests performed: 70


##### Testing in groups of 5

In [20]:
pool_sampling(100, 5)

Infected people: [1, 7, 8, 27, 47, 57, 62, 63, 78]
Number of tests performed: 55


##### Optimisation: Detect infections using the least number of tests

Created a function number_of_tests() that **returns the number of tests needed** to detect the infections when the population is split into groups for pool sampling

In [14]:
def number_of_tests(population_size, grouping_size):
    persons = list(range(population_size))
    groups = [persons[i:i+grouping_size] for i in range(0, population_size, grouping_size)]
    test_counter = 0
    
    if grouping_size == 1:
        for person in range(population_size):
                person_genome = get_genome(person)
                test_result = test(person_genome)
                test_counter += 1

    else:
        for i, group in enumerate(groups):
            combined_genome = pool(group)
            group_result = test(combined_genome)
            test_counter += 1
    
            if group_result == True:
                for person in range(grouping_size):
                    person_genome = get_genome(groups[i][person])
                    test_result = test(person_genome)
                    test_counter += 1

    return test_counter

Determine the least number of tests needed to detect infections in a population of 500

In [15]:
population_size = 500
possible_grouping_sizes = []

for x in range(1, population_size):
    if population_size % x == 0:
        possible_grouping_sizes.append(x)
        
print(possible_grouping_sizes)
        
for grouping_size in possible_grouping_sizes:
    no_tests = number_of_tests(population_size, grouping_size)
    
    if no_tests >= 500:
        continue
        
    print(f'For groups of {grouping_size}, {no_tests} tests are needed')

[1, 2, 4, 5, 10, 20, 25, 50, 100, 125, 250]
For groups of 2, 366 tests are needed
For groups of 4, 337 tests are needed
For groups of 5, 350 tests are needed
For groups of 10, 430 tests are needed
For groups of 20, 485 tests are needed
For groups of 25, 495 tests are needed


When **groups of 4 are used**, the **least number of tests (337)** are needed.

In [17]:
pool_sampling(500, 4)

Infected people: [1, 7, 8, 27, 47, 57, 62, 63, 78, 104, 105, 109, 113, 125, 128, 141, 143, 148, 151, 165, 175, 176, 181, 199, 200, 206, 218, 221, 225, 232, 242, 255, 275, 276, 295, 331, 345, 352, 368, 370, 378, 379, 380, 388, 391, 398, 402, 404, 409, 413, 414, 430, 436, 448, 449, 454, 456, 463, 476, 491, 494, 496]
Number of tests performed: 337
